In [7]:
from glob import glob
import os
routes = glob('/data/ephemeral/home/datas/text/*')
data_route = 'datas/test.csv'

In [2]:
import re
from tqdm import tqdm
def parse_wiki_file(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 문서별로 분리 (각 문서는 <doc>로 시작하고 </doc>로 끝남)
    doc_pattern = re.compile(r'<doc id="(.*?)" url="(.*?)" title="(.*?)">(.*?)</doc>', re.DOTALL)
    matches = doc_pattern.findall(content)
    
    for match in matches:
        doc_id, url, title, text = match
        documents.append({
            "id": doc_id,
            "url": url,
            "title": title,
            "text": text.strip()
        })
    
    return documents
result = []
for route in routes:
    for r in tqdm(glob(route + '/*'), desc = 'collecting...'):
        result.extend(parse_wiki_file(r))


collecting...: 100%|██████████| 100/100 [00:01<00:00, 59.71it/s]


In [8]:
import pandas as pd

df = pd.DataFrame(result)
df['doc_len'] = df['text'].apply(lambda x: len(x))

In [9]:
df = df[(df['doc_len'] > 100) & (df['doc_len'] < 2000)]

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(doc):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_text(doc)
    return chunks

df['splitted_text'] = df['text'].apply(text_split)

In [11]:
df['splitted_text'].sample(3)

205343    [북극 세계기록보관소, 북극 세계기록보관소(Arctic World Archive, ...
827610    [청주 문의문산관, 청주 문의문산관(淸州 文義文山館)은 충청북도 청주시 상당구 문의...
775180    [이지안 (미스코리아)\n\n이지안(본명: 이은희, 1977년 7월 24일~)는 대...
Name: splitted_text, dtype: object

In [15]:
data_route.split('.csv')[0]+'+' +'keyword.csv'

'datas/test+keyword.csv'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from kiwipiepy import Kiwi
from keybert import KeyBERT
from transformers import pipeline
from ast import literal_eval
from tqdm import tqdm

kw_model = KeyBERT(model='sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

# 텍스트에서 주요 키워드 추출 함수 (KeyBERT 사용)
def extract_keywords(doc, top_n=5, ngram = 2):
    kiwi = Kiwi()
    # 문장을 형태소 분석하여 명사, 형용사, 동사 등을 추출
    tokens = kiwi.tokenize(doc)
    words = [token.form for token in tokens if token.tag.startswith(('NN', 'VV', 'VA'))]  # 명사(NN), 동사(VV), 형용사(VA)만 추출

    # KeyBERT 모델을 사용하여 주요 키워드 추출
    keywords = kw_model.extract_keywords(' '.join(words), keyphrase_ngram_range=(1, ngram), stop_words=None, top_n=top_n)
    
    # 결과에서 키워드만 추출하여 리스트로 반환
    return [keyword[0] for keyword in keywords]

data = pd.read_csv(data_route)
data['dic'] = data['problems'].apply(lambda x: literal_eval(x))
data['question'] = data['dic'].apply(lambda x: x['question'])


tqdm.pandas()

# data['keywords'] = data.progress_apply(lambda x: extract_keywords(x['paragraph'] + ' ' + x['question']), axis=1)
data['keywords_1'] = data.progress_apply(lambda x: extract_keywords(x['paragraph'] + ' ' + x['question'], ngram = 1), axis=1)
# 위 코드는 ngram 2개까지 아래코드는 ngram 1개만

file_name = data_route.split('.csv')[0]+'+' +'keyword.csv'
data[['id', 'paragraph', 'problems', 'question_plus', 'keywords', 'keywords_1']].to_csv(file_name, index = False)
data = pd.read_csv(file_name)

docs = []
titles_set = set(df['title'].values)  
cnt = 0

for words in tqdm(data['keywords_1'].values):
    for word in words:
        if word in titles_set:
            docs.append(df[df['title'] == word].to_dict('records'))

# 위키 문서에서 title이 매칭되는 애들만 뽑기

flattened_docs = [item for sublist in docs for item in sublist]

df = pd.DataFrame(flattened_docs)

 88%|████████▊ | 767/869 [37:57<05:05,  3.00s/it]

In [7]:
corpus = []
for i ,item in tqdm(df.iterrows()):
    corpus += item['splitted_text']
corpus = list(filter(lambda x: len(x) > 100, corpus))
print(len(corpus))


469578it [00:20, 23134.98it/s]


725969


In [9]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os
import gc
import torch
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.schema import Document
gc.collect()
torch.cuda.empty_cache()
model_name = "intfloat/multilingual-e5-large-instruct"
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},
    encode_kwargs={'normalize_embeddings': True}
)

docs = [Document(page_content=text, metadata={"source": f"doc_{i}"}) for i, text in enumerate(corpus)]

print('making bm25..')
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2
print('making dense retrieval..')
faiss_vectorstore = FAISS.from_documents(docs, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})
ensemble_retriever = EnsembleRetriever(
retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)   



/tmp/ipykernel_155496/453807431.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  ko_embedding = HuggingFaceEmbeddings(
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


making bm25..


In [13]:
from ast import literal_eval
train = pd.read_csv('datas/test.csv')
train['dic'] = train['problems'].apply(lambda x: literal_eval(x))
train['question'] = train['dic'].apply(lambda x: x['question'])

In [19]:
idx = 42
result = ensemble_retriever.get_relevant_documents(train['paragraph'][idx] + train['question'][idx])
print(train['paragraph'][idx])
print(train['question'][idx])
print(result)

동물 병원 원장 A( 3 5세 )는 차량의 소유자로부터 공작물인   차량을 장기간 임차하여 사용하고 있다 .  A는 고객 갑이 치료를  위해 입원시킨 반려견의 재활 운동을 위해 그 차량을 운전하여  직원 B( 2 1세 )와 함께 공원으로 갔다 .  A는 차량을 주차하고 B 와  함께 반려견을 데리고 산책하던 중 B가 부주의로 반려견의 발을  밟아 상처를 입혔다 .  놀란 반려견이 지나가던 을의 다리를 물어  2주간의 치료를 요하는 상처를 입혔고,  을은 이로 인해 정신적 으로도 큰 충격을 받았다 .  그 사이 A가 주차해 두었던 차량에서  불이 났고,  이로 인해 옆에 주차되어 있던 병 소유 차량이 파손 되는 재산상 손해가 발생하였다 .  사고 조사 결과,  화재의 원인은  차량에 대한 비전문가인 소유자가 해당 차량을 직접 수리하여   발생한 보존상의 하자에 의한 것으로 밝혀졌다 .
다음 사례에 대한 법적 판단으로 옳은 것은 ?
[Document(metadata={'source': 'doc_415645'}, page_content='(7) WRITE-ITEM C .\n (8) READ-ITEM A ; IF END OF DATA GO TO OPERATION 14 .\n (9) JUMP TO OPERATION 1 .\n(10) READ-ITEM B ; IF END OF DATA GO TO OPERATION 12 .\n(11) JUMP TO OPERATION 1 .\n(12) SET OPERATION 9 TO GO TO OPERATION 2 .\n(13) JUMP TO OPERATION 2 .\n(14) TEST PRODUCT-NO (B) AGAINST ZZZZZZZZZZZZ ; IF EQUAL GO TO OPERATION 16 ;\n OTHERWISE GO TO OPERATION 15 .\n(15) REWIND B .\n(16) CLOSE-OUT FILES C ; D .\n이 샘플에는 오직 프로그램의 실행문들 섹션만 포함되어 있다. 레코드 필드 와 는 섹션에 정의

In [184]:
from tqdm import tqdm

def generate_hint(row):
    query = row['question'] + ' ' + row['question']
    relevant_docs = ensemble_retriever.get_relevant_documents(query)
    
    # `hint` 문자열 생성
    hint = ''
    for i, relevant_doc in enumerate(relevant_docs):
        hint += f"참고문서{i} : {relevant_doc.page_content}\n"
        if i == 2:
            break  # 3개까지만
    return hint

# tqdm 적용 및 `apply`를 사용하여 새 열 생성
tqdm.pandas()
train['hint'] = train.progress_apply(generate_hint, axis=1)


100%|██████████| 2031/2031 [13:45<00:00,  2.46it/s]


In [186]:
d = pd.read_csv('datas/train+klue.csv')
d['hint'] = train['hint']

In [187]:
print(d['hint'][4])

참고문서0 : 이 때에 자기의 의로우심을 나타내사 자기도 의로우시며 또한 예수 믿는 자를 의롭다 하려 하심이니라 그런즉 자랑할 데가 어디뇨 있을 수가 없느니라 무슨 법으로냐 행위로냐 아니라 오직 믿음의 법으로니라 그러므로 사람이 의롭다 하심을 얻는 것은 율법의 행위에 있지 않고 믿음으로 되는줄 우리가 인정하노라”
참고문서1 : 이렇게 사찰의 건축물로서 지어졌던 탑도 있지만 민간에서 단순하게 주변에 있던 돌을 쌓아 올려 만든 돌탑도 있다. 소박하게 만들어진 이런 돌탑들은 토속신앙과 관련이 깊어 서낭신을 모시는 서낭당과 가까이 있는 경우가 많았으며, 기복의 용도로 이용되었다. 한국의 불탑은 중국과 일본에 비해 그 규모가 작은 편이다. 이는 오랜 전란으로 인한 문화재의 훼손이 잦았고, 조선의 숭유억불 정책으로 인해 의도적으로 사찰을 파괴하는 경우가 많았기 때문이다.



In [189]:
d[['id', 'paragraph', 'problems', 'question_plus', 'klue', 'hint']].to_csv(file_name)